# UTM Coordinate Conversor 
This script receives UTM Coordinates and corvert them to latitude and longitude.

### The csv file must be in this format:
X, Y


coord, coord


coord, coord


coord, coord


..., ...



In [54]:
import csv
import pyproj
import folium
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon

# Here you put the file path
FILE = 'Coordenadas2.csv'

_projections = {}

print(pd.read_csv(FILE))

         X        Y
0   478725  2842846
1   477911  2842246
2   472552  2842216
3   469381  2842512
4   467130  2843643
5   463342  2847006
6   460350  2849900
7   460601  2857001
8   458800  2859580
9   458460  2865155
10  457323  2870038
11  457272  2872766
12  460354  2879921
13  463885  2883841
14  468005  2884366


In [53]:
def getUtmCoordinatesFromCsv(file):
    with open(file, 'r') as csvfile:
        csv_reader = csv.reader(csvfile)
        next(csv_reader)
        utmCoordinates = [[]]
        a = 0
        for line in csv_reader:
            for coords in line:
                utmCoordinates[a].append(int(coords))
            utmCoordinates.append([])
            a += 1
    utmCoordinates.pop()
    return utmCoordinates

In [29]:
UTM_COORDINATES = getUtmCoordinatesFromCsv(FILE)
print(UTM_COORDINATES)

[[478725, 2842846], [477911, 2842246], [472552, 2842216], [469381, 2842512], [467130, 2843643], [463342, 2847006], [460350, 2849900], [460601, 2857001], [458800, 2859580], [458460, 2865155], [457323, 2870038], [457272, 2872766], [460354, 2879921], [463885, 2883841], [468005, 2884366]]


In [2]:
def unproject(z, l, x, y):
    if z not in _projections:
        _projections[z] = pyproj.Proj(proj='utm', zone=z, ellps='WGS84')
    if l < 'N':
        y -= 10000000
    lng, lat = _projections[z](x, y, inverse=True)
    return (lng, lat)

In [33]:
def getLatitudeLongitude(coordinatesUTM):
    l = "R"
    z = 12
    longList = []
    latList = []
    for coordinates in coordinatesUTM:
        lng, lat = unproject(z, l, coordinates[0], coordinates[1])
        latList.append(lat)
        longList.append(lng)
    return latList, longList

In [34]:
latList, longList = getLatitudeLongitude(UTM_COORDINATES)
print(latList, longList)

[25.703318627846304, 25.697888503295903, 25.697527186163537, 25.70013738785999, 25.710301729263374, 25.740579931614143, 25.766634705538873, 25.830762790559334, 25.854000975905958, 25.90433231837708, 25.948391440787116, 25.973022848588737, 26.03771742888544, 26.07320599577715, 26.07804372042711] [-111.21205477917985, -111.220158167176, -111.27356956178869, -111.30518085731181, -111.32764434718007, -111.36549493141014, -111.39541234770167, -111.39312073994799, -111.41117114017517, -111.41474001433164, -111.42625027689799, -111.42684850236161, -111.39627699269832, -111.36109234196064, -111.31991234812486]


In [36]:
def dataFrame(latList, longList):
    dataFrame = pd.DataFrame(np.c_[tuple(latList), tuple(longList)], columns=[ 'Lat', 'Lon'])
    return dataFrame

In [47]:
latitudeLongitudeDataFrame = dataFrame(latList, longList)
latitudeLongitudeDataFrame.to_csv('latlon.csv', index=False)
print(pd.read_csv('latlon.csv'))

          Lat         Lon
0   25.703319 -111.212055
1   25.697889 -111.220158
2   25.697527 -111.273570
3   25.700137 -111.305181
4   25.710302 -111.327644
5   25.740580 -111.365495
6   25.766635 -111.395412
7   25.830763 -111.393121
8   25.854001 -111.411171
9   25.904332 -111.414740
10  25.948391 -111.426250
11  25.973023 -111.426849
12  26.037717 -111.396277
13  26.073206 -111.361092
14  26.078044 -111.319912


In [72]:
polygon_geom = Polygon(zip(longList, latList))
crs = {'init': 'epsg:4326'}
polygon = gpd.GeoDataFrame(index=[0], crs=crs, geometry=[polygon_geom])       
print(polygon.geometry)

polygon.to_file(filename='polygon.geojson', driver='GeoJSON')
polygon.to_file(filename='polygon.shp', driver="ESRI Shapefile")

0    POLYGON ((-111.21205 25.70332, -111.22016 25.6...
Name: geometry, dtype: geometry


In [79]:
m = folium.Map(location=[25.918145, -111.274523], zoom_start=9.5)
folium.GeoJson('polygon.geojson', name="geojson").add_to(m)
m